In [1]:
import pandas as pd 
import numpy as np 
import sys 
import os 
import importlib
import copy

In [2]:
import Protein_Input_funcions as pif
import ReadingFasta_JL as ReadingFasta
import FixedClassificationModel_JL as FixedClassificationModel
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [3]:
importlib.reload(FixedClassificationModel)

<module 'FixedClassificationModel_JL' from '/mnt/af3ff5c3-2943-4972-8c3a-6b98174779b7/Justice/Chaperone_Analysis/OR_ML/Rtp1s_RF/FixedClassificationModel_JL.py'>

In [6]:
matrix = pd.read_csv('./AlphaFold/xyz_enhance_matrix.csv')
OR_dependency = pd.read_csv('./OR_RTP_dependency.csv', 
                            index_col=0)[['id','Normalized_PE_(RTP1S/raw)']].drop_duplicates('id')\
                            .set_index('id')

In [7]:
# Inserts normalized_PE to the 3rd column in matrix
matrix.insert(2,"Normalized_PE", OR_dependency.reset_index()['Normalized_PE_(RTP1S/raw)'])

In [10]:
matrix

,id,enhance,Normalized_PE,25_C_x,25_C_y,25_C_z,25_CA_x,25_CA_y,25_CA_z,25_N_x,...,89_CA_z,89_N_x,89_N_y,89_N_z,89_O_x,89_O_y,89_O_z,90_C_x,90_C_y,90_C_z
0,Olfr649,1,3.610785,25.563518,-4.154193,0.537515,26.853694,-3.516590,1.047625,27.671076,...,-9.501947,23.075911,-9.290980,-8.153995,24.472024,-7.898104,-11.206724,23.151720,-4.962594,-10.515114
1,Olfr618,0,1.200599,27.665911,-3.783908,1.723106,28.554943,-3.063699,0.690824,27.816769,...,-4.554676,21.679727,-9.496581,-4.437274,24.370635,-11.382724,-5.789047,23.700653,-9.314173,-9.015133
2,Olfr666,0,1.396552,25.263184,-2.609801,0.724571,26.585811,-2.013219,1.238045,27.424313,...,-9.339453,22.411036,-8.692103,-7.989804,23.631875,-7.921017,-11.327901,22.493656,-5.160381,-11.541808
3,Olfr654,0,1.102167,19.907107,16.892373,-19.636828,20.343711,17.668423,-20.881501,21.205100,...,-8.637408,-2.566475,-5.178184,-7.456887,0.159261,-4.790043,-9.605506,0.278787,-1.843004,-9.354504
4,Olfr657,0,0.796421,26.804558,-0.487396,-0.732470,26.750054,0.960334,-1.234307,25.643494,...,-10.767177,15.297882,-10.314373,-9.503973,18.347742,-9.993534,-11.253381,19.453462,-8.262900,-9.025919
5,Olfr556,1,14.755927,25.945317,-1.408705,0.923833,25.353593,-0.225225,0.141355,24.887309,...,-2.802477,18.443160,-4.467465,-3.939000,21.030230,-6.333985,-2.253272,20.843618,-8.662488,-4.840067
6,Olfr544,1,3.560356,30.702541,-6.021952,-2.378727,30.221502,-4.634519,-2.844029,29.927807,...,-3.971445,22.076630,-8.759782,-4.643335,25.561554,-9.538637,-4.682475,24.800351,-10.122499,-8.160582
7,Olfr521,0,0.891156,27.009302,-4.731731,-2.499952,27.449100,-3.428764,-1.820270,26.284756,...,-9.947934,15.921692,-7.888423,-9.953773,19.297849,-7.198041,-9.405346,19.579486,-9.405185,-7.138831
8,Olfr638,1,3.561151,23.708143,0.345006,-2.901990,22.336859,-0.205432,-2.497042,21.852234,...,-6.527041,14.039851,-5.912641,-7.735075,16.968369,-7.100964,-6.353381,16.898642,-9.695940,-8.059885
9,Olfr577,0,1.482332,31.399829,-4.595508,-0.424107,32.202528,-3.850529,-1.509797,31.496565,...,-6.070857,23.242020,-8.697739,-5.815275,25.635658,-10.768632,-7.433683,25.277893,-8.276253,-10.297858


In [9]:
matrix.to_csv('./AlphaFold/xyz_matrix.csv')

In [75]:
# Converts the data table of coordinates to list of list 
X = matrix.drop(['id', 'enhance', "Normalized_PE"], axis=1).values.tolist()
# y = matrix['enhance'].tolist()
y = matrix['Normalized_PE'].tolist()

feature = matrix.drop(['id', 'enhance'], axis=1).columns.tolist()


# if (y.count(1)/y.count(0) > 1.5) | (y.count(0)/y.count(1) > 1.5):
#     BALANCE = False
# else: 
#     BALANCE = True 

In [76]:
X_train, X_test, y_train, y_test = \
FixedClassificationModel.train_test_split(X, y, test_size = .25, random_state = 18)

In [83]:
pca = PCA(n_components = 10)
pca.fit(X)
X_reduced = pca.transform(X)

In [84]:
pc_label = []
for i, var in enumerate(pca.explained_variance_ratio_):
    pc_label.append("PC{}_({}%)".format(i, round(var*100, 2)))
    
pca_df = pd.DataFrame(
    data=X_reduced,
    columns=pc_label,
    index=matrix.index
)

In [96]:
from sklearn.preprocessing import minmax_scale
minmax_scale(arr)

array([2.17036656e-02, 3.11691956e-03, 4.62805862e-03, 2.35783997e-03,
       0.00000000e+00, 1.07652164e-01, 2.13147648e-02, 7.30579002e-04,
       2.13208979e-02, 5.28957547e-03, 3.37049235e-03, 1.76274629e-03,
       7.50853696e-02, 3.55946930e-03, 3.35448880e-03, 7.91478101e-02,
       1.58508582e-01, 3.25996773e-02, 3.94148462e-02, 1.82530847e-03,
       3.72489776e-02, 1.49326996e-01, 9.64423563e-02, 3.66553566e-03,
       1.06259473e-02, 2.92863429e-02, 1.48310963e-01, 9.15112890e-03,
       2.58154448e-03, 7.04310350e-01, 1.00000000e+00, 4.19257551e-01,
       3.20758200e-03, 1.30299089e-01, 2.23872191e-03, 1.18003294e-03,
       2.51787527e-03, 8.16986604e-02, 7.44975234e-04, 4.27188846e-02,
       3.90765948e-03, 1.72413211e-02, 2.30873042e-01, 1.50487463e-03,
       2.46506597e-03, 2.65393294e-03, 6.12276691e-02, 3.36686423e-03,
       6.78647668e-02, 2.31232926e-02])

In [98]:
import plotly.express as px

fig = px.scatter(pca_df, x=pca_df.columns[0], 
                         y=pca_df.columns[1], 
                 color=matrix.enhance,
                 hover_data=[matrix.id],
                title = "")
fig.update_traces(marker=dict(size=15))
fig.show()